In [1]:
import sys

__import__('pysqlite3')

# Replace sqlite3 with pysqlite3
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
! pip install -U langchain openai chromadb langchain-experimental # (newest versions required for multi-modal)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 12.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.7/173.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.0
    Uninstalling langsmith-0.1.0:
      Successfully uninstalled langsmith-0.1.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.28
    Uninstalling langchain-core-0.1.28:
      Successfully uninstalled langchain-core-0.1.28
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.9
    Uninstalling langchain-0.1.9:
      Successfully uninstalled langchain-0.1.9
  Attempting uninstall: chromadb
    Found existing installation: chromadb

In [3]:
# lock to 0.10.19 due to a persistent bug in more recent versions
! pip install "unstructured[all-docs]==0.10.19" pillow pydantic lxml pillow matplotlib tiktoken open_clip_torch torch

  Using cached unstructured-0.10.19-py3-none-any.whl.metadata (24 kB)
Using cached unstructured-0.10.19-py3-none-any.whl (1.6 MB)
  Attempting uninstall: unstructured
    Found existing installation: unstructured 0.12.5
    Uninstalling unstructured-0.12.5:
      Successfully uninstalled unstructured-0.12.5


Extract

In [4]:
# Folder with pdf and extracted images
path = "/home/vqa/masterthesis/ourproject/multimodal_rag/extracted_data/photos/"

In [5]:
# Extract images, tables, and chunk text
from unstructured.partition.pdf import partition_pdf

raw_pdf_elements = partition_pdf(
    filename=path + "LLaVA.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# Categorize text elements by type
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

In [18]:
print(texts)

['Visual Instruction Tuning\n\nHaotian Liu1∗, Chunyuan Li2∗, Qingyang Wu3, Yong Jae Lee1\n\n1University of Wisconsin–Madison\n\nTUniversity of Wisconsin-Madison ?Microsoft Research *Columbia University\n\nhttps://llava-vl.github.io\n\nhttps://llava-vl.github.io\n\narXiv:2304.08485v2 [cs.CV] 11 Dec\n\n3 2 0 2 c e D 1 1\n\n2 v 5 8 4 8 0 . 4 0 3 2 : v i X r a\n\n1\n\nAbstract\n\nInstruction tuning large language models (LLMs) using machine-generated instruction-following data has been shown to improve zero-shot capabilities on new tasks, but the idea is less explored in the multimodal field. We present the first attempt to use language-only GPT-4 to generate multimodal language-image instruction-following data. By instruction tuning on such generated data, we in- troduce LLaVA: Large Language and Vision Assistant, an end-to-end trained large multimodal model that connects a vision encoder and an LLM for general- purpose visual and language understanding. To facilitate future research on v

Embeddings

In [8]:

import os
import uuid

import chromadb
import chromadb.config
import numpy as np
from langchain_community.vectorstores import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from PIL import Image as _PILImage

# Create chroma
vectorstore = Chroma(
    collection_name="mm_rag_clip_photos", embedding_function=OpenCLIPEmbeddings()
)

# Get image URIs with .jpg extension only
image_uris = sorted(
    [
        os.path.join(path, image_name)
        for image_name in os.listdir(path)
        if image_name.endswith(".jpg")
    ]
)

# Add images
vectorstore.add_images(uris=image_uris)

# Add documents
vectorstore.add_texts(texts=texts)

# Make retriever
retriever = vectorstore.as_retriever()


In [14]:
import base64
import io
from io import BytesIO

import numpy as np
from PIL import Image


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string.

    Args:
    base64_string (str): Base64 string of the original image.
    size (tuple): Desired size of the image as (width, height).

    Returns:
    str: Base64 string of the resized image.
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def is_base64(s):
    """Check if a string is Base64 encoded"""
    try:
        return base64.b64encode(base64.b64decode(s)) == s.encode()
    except Exception:
        return False


def split_image_text_types(docs):
    """Split numpy array images and texts"""
    images = []
    text = []
    for doc in docs:
        doc = doc.page_content  # Extract Document contents
        if is_base64(doc):
            # Resize image to avoid OAI server error
            images.append(
                resize_base64_image(doc, size=(250, 250))
            )  # base64 encoded str
        else:
            text.append(doc)
    return {"images": images, "texts": text}

In [15]:
from operator import itemgetter

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama


def prompt_func(data_dict):
    # Joining the context texts into a single string
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        image_message = {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{data_dict['context']['images'][0]}"
            },
        }
        messages.append(image_message)

    # Adding the text message for analysis
    text_message = {
        "type": "text",
        "text": (
            "As an expert art critic and historian, your task is to analyze and interpret images, "
            "considering their historical and cultural significance. Alongside the images, you will be "
            "provided with related text to offer context. Both will be retrieved from a vectorstore based "
            "on user-input keywords. Please use your extensive knowledge and analytical skills to provide a "
            "comprehensive summary that includes:\n"
            "- A detailed description of the visual elements in the image.\n"
            "- The historical and cultural context of the image.\n"
            "- An interpretation of the image's symbolism and meaning.\n"
            "- Connections between the image and the related text.\n\n"
            f"User-provided keywords: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)

    return [HumanMessage(content=messages)]


# model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)
# model = ChatOllama(model="llava")
model = ChatOllama(model="llama2:7b-chat")


# RAG pipeline
chain = (
    {
        "context": retriever | RunnableLambda(split_image_text_types),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(prompt_func)
    | model
    | StrOutputParser()
)

In [16]:
from IPython.display import HTML, display


def plt_img_base64(img_base64):
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'

    # Display the image by rendering the HTML
    display(HTML(image_html))


docs = retriever.get_relevant_documents("Woman with children", k=10)
for doc in docs:
    if is_base64(doc.page_content):
        plt_img_base64(doc.page_content)
    else:
        print(doc.page_content)

8User  Do you know who drew this paintingThe painting depicts a woman, commonly believed to be Mona Lisa, the famous artwork by Leonardo da Vinci. It is a portrait painting thatshowcases the woman's enigmatic smile and has become one of the most famous and iconic art pieces in the world. The original work isdisplayed in the Louvre Museum in Paris, and it is known for its intricate details, use of oil paint, and the artist's innovative techniques thatLLaVA contributed to its enduring appeal and mystery.  Start a new conversation, and the history is cleared.  Do you know who drew tThe painting depicts a dog in a humorous situation, where it appears to be wearing a woman's dress and standing in front of a landscape.Although the artist of the painting is not explicitly identified, it could be a reference to the famous artist Da Vinci, known for his artistic works,including the Mona Lisa. The painting is likely a creative and comical take on the traditional portrait style, replacing a human

In [17]:
chain.invoke("Explain the chicken nugget picture.")

'As an art critic and historian, I can provide a detailed analysis of the chicken nugget meme. The image depicts a tray of cooked chicken nuggets arranged to resemble a map of the world, with each nugget representing a different continent or island. The meme pokes fun at the idea that something as mundane and earthly as chicken nuggets could be seen as a representation of the entire world.\n\nThe visual elements in the image include the arrangement of the chicken nuggets, which form a crude map of the world. Each nugget is distinctly shaped to represent a particular continent or island, with some nuggets even featuring tiny details like mountains or oceans. The image also features a small amount of text at the bottom, which reads "I mean, it\'s not the real Earth, but how beautiful it is all." This adds a layer of humor to the meme by acknowledging the absurdity of using chicken nuggets to represent the entire world.\n\nThe historical and cultural context of the image is important to u